In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e1/sample_submission.csv
/kaggle/input/playground-series-s5e1/train.csv
/kaggle/input/playground-series-s5e1/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s5e1/train.csv")
train.head()

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [3]:
train.isnull().sum()

id             0
date           0
country        0
store          0
product        0
num_sold    8871
dtype: int64

In [4]:
train = train.dropna()

In [5]:
train['date'] = pd.to_datetime(train['date'])
train.dtypes

id                   int64
date        datetime64[ns]
country             object
store               object
product             object
num_sold           float64
dtype: object

In [6]:
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day

In [7]:
train.head()

,id,date,country,store,product,num_sold,year,month,day
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0,2010,1,1
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0,2010,1,1
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0,2010,1,1
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0,2010,1,1
5,5,2010-01-01,Canada,Stickers for Less,Holographic Goose,300.0,2010,1,1


In [8]:
train = train.drop("date", axis = 1)

In [9]:
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

In [10]:
for i in train.columns:
    if train[i].dtype =='object':
        encoder = LabelEncoder()
        train[i] = encoder.fit_transform(train[i])

In [11]:
train.head()

,id,country,store,product,num_sold,year,month,day
1,1,0,0,1,973.0,2010,1,1
2,2,0,0,2,906.0,2010,1,1
3,3,0,0,3,423.0,2010,1,1
4,4,0,0,4,491.0,2010,1,1
5,5,0,2,0,300.0,2010,1,1


In [12]:
scalar = StandardScaler()
train_scaled = scalar.fit_transform(train)

In [13]:
train_scaled = pd.DataFrame(train_scaled, columns=train.columns)

In [14]:
Y = train['num_sold']
X = train.drop("num_sold", axis = 1)

In [15]:
model = RandomForestRegressor(random_state = 42)

In [16]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

In [17]:
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,  # 5-Fold Cross Validation
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

In [18]:
test = pd.read_csv("/kaggle/input/playground-series-s5e1/test.csv")

test.head()

,id,date,country,store,product
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode


In [19]:
test.isnull().sum()

id         0
date       0
country    0
store      0
product    0
dtype: int64

In [20]:
train

,id,country,store,product,num_sold,year,month,day
1,1,0,0,1,973.0,2010,1,1
2,2,0,0,2,906.0,2010,1,1
3,3,0,0,3,423.0,2010,1,1
4,4,0,0,4,491.0,2010,1,1
5,5,0,2,0,300.0,2010,1,1
...,...,...,...,...,...,...,...,...
230125,230125,5,1,0,466.0,2016,12,31
230126,230126,5,1,1,2907.0,2016,12,31
230127,230127,5,1,2,2299.0,2016,12,31
230128,230128,5,1,3,1242.0,2016,12,31
